In [1]:
import glob
import pandas as pd
import numpy as np

file_paths = [r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\abnormal\train",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\abnormal\test",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\normal\train",\
             r"C:\Users\호재\Desktop\대학\3학년 2학기\기계학습론\data\IC_PBL\normal\test"]

dataframes_train_abn = []
dataframes_test_abn  = []
dataframes_train_n  = []
dataframes_test_n  = []

df_list = [dataframes_train_abn, dataframes_test_abn, dataframes_train_n, dataframes_test_n]

for i in range(len(file_paths)) :
    files = glob.glob(file_paths[i] + "\\**\\*.csv", recursive=True)
    
    for file_path in files:
        df = pd.read_csv(file_path, encoding='cp949', header=None)
        df_list[i].append(df)


In [2]:
dataframes_train_abn_0 = []
dataframes_test_abn_0 = []
dataframes_train_n_0  = []
dataframes_test_n_0 = []

df_list_0 = [dataframes_train_abn_0, dataframes_test_abn_0, dataframes_train_n_0, dataframes_test_n_0]

for j in range(len(df_list)):
    df_list_0[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0], 256)])

dataframes_train_abn_25 = []
dataframes_test_abn_25 = []
dataframes_train_n_25  = []
dataframes_test_n_25 = []

df_list_25 = [dataframes_train_abn_25, dataframes_test_abn_25, dataframes_train_n_25, dataframes_test_n_25]

for j in range(len(df_list)):
    df_list_25[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0]-64, 256-64)])

dataframes_train_abn_50 = []
dataframes_test_abn_50 = []
dataframes_train_n_50  = []
dataframes_test_n_50 = []

df_list_50 = [dataframes_train_abn_50, dataframes_test_abn_50, dataframes_train_n_50, dataframes_test_n_50]

for j in range(len(df_list)):
    df_list_50[j].extend([df[i:i+256] for df in df_list[j] for i in range(0, df.shape[0]-128, 256-128)])


In [3]:
import pandas as pd
import numpy as np

def Fourier(df_list):
    amp_df_list = []
    for df in df_list:
        time = df[0].values
        amplitude = df[1].values
        freq = np.fft.rfftfreq(len(time), np.diff(time)[0])  # 주파수 축 계산
        fft_values = np.fft.rfft(amplitude)  # 푸리에 변환

        # 푸리에 변환 결과에서 진폭값만 추출하고 데이터프레임으로 변환
        amp_df = pd.DataFrame({'Amplitude': np.abs(fft_values)})
        amp_df_list.append(amp_df)
        
    return amp_df_list

def transpose_df_list(df_list):
    reform_list = []
    for i in range(len(df_list)):
        df = df_list[i].transpose()
        reform_list.append(df)
    return reform_list

def create_train_test(df_list):
    a_list = [pd.DataFrame() for _ in range(4)]
    for i in range(4):
        a_list[i] = pd.concat(df_list[i], ignore_index=True)

    train_abn, test_abn, train_n, test_n = a_list
    train_abn['target'] = 1
    test_abn['target'] = 1
    train_n['target'] = 0
    test_n['target'] = 0
    
    train = pd.concat([train_abn, train_n], ignore_index=True)
    test = pd.concat([test_abn, test_n], ignore_index=True)
    
    return train, test

# 함수 적용
df_lists = [df_list_0, df_list_25, df_list_50]
train_test_lists = []

for df_list in df_lists:
    for i in range(4):
        df_list[i] = Fourier(df_list[i])
        
    for i in range(4):
        df_list[i] = transpose_df_list(df_list[i])
        
    train_test_lists.append(create_train_test(df_list))

train_0, test_0 = train_test_lists[0]
train_25, test_25 = train_test_lists[1]
train_50, test_50 = train_test_lists[2]


In [10]:
train_50

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,target
0,0.48937,0.754067,0.721972,1.105029,0.537482,1.813463,1.504164,1.096414,1.734573,0.883342,...,1.627999,1.891413,2.504647,3.790013,0.740094,1.527207,1.174129,0.719002,0.49901,1
1,0.52770,0.541578,0.761056,0.344304,0.152391,0.617480,0.688849,0.090547,0.715986,1.161087,...,3.974455,1.165119,4.323760,2.158711,1.255104,1.590903,1.326946,0.762048,0.92112,1
2,0.70052,0.631650,0.766651,0.966008,1.400714,1.154523,0.788774,1.101803,1.428216,1.102083,...,2.967666,2.339524,3.755707,3.427759,1.249829,1.355175,0.933543,0.892415,1.22814,1
3,0.41263,0.436271,0.317496,0.929204,0.840909,0.844027,0.681465,0.983148,0.380608,1.033082,...,0.875341,0.521724,3.017952,4.099280,0.970995,0.527120,0.132278,1.022474,0.03827,1
4,0.02881,0.527805,0.288918,0.623191,0.405082,0.695788,0.631081,0.231085,0.270639,0.862010,...,2.201313,1.401933,2.393469,3.829343,1.208291,0.262299,0.571871,0.450383,0.49905,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43808,0.78672,0.748952,0.709291,1.130966,0.591530,0.877762,0.566130,1.543020,1.850425,1.358638,...,0.296316,0.897958,0.519776,0.896798,0.311695,0.189021,1.133149,0.614772,0.92114,0
43809,1.20891,0.996321,0.966977,0.475782,1.112874,1.010350,0.764853,1.310861,1.425554,1.447355,...,0.325480,1.008079,0.473147,0.601169,0.154571,0.420851,0.929524,1.029857,0.42219,0
43810,0.24935,0.304071,0.231421,0.850941,0.872591,1.328186,1.115613,0.742598,1.430128,1.344100,...,0.605557,0.515168,0.928299,0.403858,0.993405,0.265181,0.870892,0.656376,0.07673,0
43811,0.82537,0.625223,1.044647,0.743304,0.487835,1.299951,1.135163,1.297731,0.445318,1.820537,...,0.491705,0.496765,0.319917,0.411757,0.630737,0.629251,0.800845,0.773662,0.61405,0


In [11]:
test_50

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,128,target
0,0.19194,0.210006,0.391758,0.581497,0.874277,0.623490,0.218901,1.088837,1.591278,0.772905,...,2.134021,1.438916,0.490218,1.415163,4.815775,1.924191,1.570018,1.030902,1.72716,1
1,0.34547,0.241019,0.342496,0.156686,0.478969,0.615609,0.405243,0.776744,0.379340,0.538131,...,0.885368,0.711911,0.835516,1.726974,4.994260,1.213866,1.469853,0.780672,0.57571,1
2,0.72926,0.873587,0.812466,1.068905,1.094146,1.329997,0.842467,0.958614,1.106494,0.094227,...,1.185059,0.927185,0.778212,1.226319,5.266012,0.658263,1.025876,0.884462,0.49898,1
3,0.49897,0.164743,0.500343,0.119167,0.612869,0.689399,0.989933,0.529429,0.860493,0.855577,...,1.541940,0.423688,0.414390,1.021949,5.355921,0.174727,0.506255,0.997971,0.03839,1
4,0.69082,0.379602,0.783784,0.476003,1.027064,1.196473,0.681820,0.706149,1.091667,0.637033,...,0.850400,0.167242,0.677946,1.413754,6.408808,1.626650,0.939279,1.273211,1.07470,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14982,0.10554,0.230171,0.244599,0.330824,0.505637,1.376334,1.614293,1.231687,0.912351,0.688792,...,3.769557,0.385472,0.977815,0.641427,0.799339,0.686546,0.664267,1.069546,1.49694,0
14983,0.22067,0.125276,0.293426,0.514727,0.305144,1.706292,0.779616,1.264492,0.395264,1.074267,...,2.977128,0.836831,1.543351,0.363733,1.164699,0.580970,1.268200,1.415845,2.14941,0
14984,0.16310,0.555696,0.554851,0.648878,0.885458,1.974532,0.525583,0.670838,1.804562,0.509460,...,2.697387,1.347154,1.780347,1.831611,0.458441,0.447200,0.224293,2.188568,1.15144,0
14985,0.70044,0.957835,1.156810,1.092646,0.969673,1.193861,1.421963,1.513769,1.766504,1.331634,...,4.385881,0.653784,0.500474,1.005363,0.519931,1.492293,0.877471,0.425176,0.69098,0


In [7]:
train_0.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_0_fft.csv", encoding= 'cp949', index=False)
test_0.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_0_fft.csv", encoding= 'cp949', index=False)
train_25.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_25_fft.csv", encoding= 'cp949', index=False)
test_25.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_25_fft.csv", encoding= 'cp949', index=False)
train_50.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/train_50_fft.csv", encoding= 'cp949', index=False)
test_50.to_csv(r"C:/Users/호재/Desktop/대학/3학년 2학기/기계학습론/data/IC_PBL/test_50_fft.csv", encoding= 'cp949', index=False)